In [27]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re

In [3]:
seoul = gpd.read_file('tl_emd_seoul_4326/tl_emd._seoul_4326.shp')

In [20]:
with pd.ExcelWriter('data/seoul_dong_polygon.xlsx') as writer:  
    seoul.to_excel(writer, sheet_name='Sheet_name_1',startrow = 3, startcol = 3)

In [100]:
temp2 = temp.drop_duplicates(['gu','bub'])

In [102]:
temp2.rename(columns = {'bub':'emd_kor_nm'},inplace = True)

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [105]:
seoul2 = pd.merge(seoul,temp2,on = 'emd_kor_nm',how = 'left')

In [126]:
temp2[temp2['emd_kor_nm'] == '신사동']

,gu,hang,emd_kor_nm
0,은평구,신사제1동,신사동
0,강남구,신사동,신사동


In [120]:
sin = seoul[seoul['emd_kor_nm'] == '신정동']
sin2 = seoul[seoul['emd_kor_nm'] == '신사동']

In [125]:
sin2

,emd_cd,emd_eng_nm,emd_kor_nm,geometry
124,11680107,Sinsa-dong,신사동,"MULTIPOLYGON (((127.02191 37.51711, 127.01953 ..."
251,11380109,Sinsa-dong,신사동,"POLYGON ((126.90211 37.60374, 126.90492 37.603..."


In [116]:
seoul2 = seoul[seoul['emd_kor_nm'] != '신정동']
seoul3 = seoul2[seoul2['emd_kor_nm'] != '신사동']

In [119]:
seoul4 = pd.merge(seoul3,temp2,on = 'emd_kor_nm',how = 'left')

In [122]:
seoul4.head()

,emd_cd,emd_eng_nm,emd_kor_nm,geometry,gu,hang
0,11110152,Bongik-dong,봉익동,"POLYGON ((126.99254 37.57385, 126.99409 37.572...",종로구,종로1234가동
1,11110153,Donui-dong,돈의동,"POLYGON ((126.99101 37.57336, 126.99178 37.570...",종로구,종로1234가동
2,11560123,Mullae-dong 5(o)-ga,문래동5가,"POLYGON ((126.88453 37.51788, 126.88881 37.517...",영등포구,문래동
3,11110141,Anguk-dong,안국동,"POLYGON ((126.98377 37.57947, 126.98426 37.579...",종로구,삼청동
4,11110142,Sogyeok-dong,소격동,"POLYGON ((126.98112 37.58153, 126.98234 37.579...",종로구,삼청동


In [127]:
sin['gu'],sin['hang'] = ['마포구','양천구'], ['신수동','신정제1동']
sin2['gu'],sin2['hang'] = ['강남구','은평구'], ['신사동','신사제1동']

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [129]:
seoul_fin = pd.concat([seoul4,sin,sin2],axis = 0)

In [132]:
seoul_fin.index = range(len(seoul_fin))

In [136]:
seoul_notnull = seoul_fin.iloc[seoul_fin.isnull()[seoul_fin.isnull()['gu'] == False].index.tolist()]

In [137]:
seoul_null = seoul_fin.iloc[seoul_fin.isnull()[seoul_fin.isnull()['gu'] == True].index.tolist()]

In [139]:
seoul_null['gu'] = ['종로구']*2 + ['영등포구'] * 8 + ['중구'] * 3 

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [141]:
seoul_fin_fin = pd.concat([seoul_notnull,seoul_null],axis = 0)

In [142]:
seoul_fin_fin.to_file("seoul_total_dong_polygon.shp",encoding = 'utf8')

In [149]:
type(seoul_fin_fin)

geopandas.geodataframe.GeoDataFrame

# 법정동 크롤러

In [25]:
base_url = 'https://www.seoul.go.kr/seoul/autonomy_sub.do'

In [30]:
browser = webdriver.Chrome('/Users/sinmin-yong/GitHub/공모전/도서빅데이터/chromedriver')
browser.get(base_url)
time.sleep(1.5)

In [35]:
gu_ls = browser.find_elements_by_xpath('//*[@class = "tabs2 boxtabs"]/ul/li')[1:]

In [39]:
gu_ls[0].click()

In [43]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [44]:
table_ls = soup.find_all('table',{'class':'tstyle04'})

In [45]:
table_ls[0]

<table class="tstyle04">
<caption class="sr-only">종로구(<i>鐘路區</i>)</caption>
<colgroup>
<col style="width:25%"/>
<col/>
</colgroup>
<thead>
<tr>
<th scope="col"><i>行 政 洞</i></th>
<th scope="col"><i>法 定 洞</i></th>
</tr>
</thead>
<tbody>
<tr>
<td>청운효자동
                                        (<i>淸雲孝子洞</i>)</td>
<td>청운동(<i>淸雲洞</i>), 신교동(<i>新橋洞</i>), 궁정동(<i>宮井洞</i>), 세종로(<i>世宗路</i>), 효자동(<i>孝子洞</i>), 창성동(<i>昌成洞</i>), 통인동(<i>通仁洞</i>), 누상동(<i>樓上洞</i>), 누하동(<i>樓下洞</i>), 옥인동(<i>玉仁洞</i>)</td>
</tr>
<tr>
<td>사직동(<i>社稷洞</i>)</td>
<td>사직동(<i>社稷洞</i>), 체부동(<i>體府洞</i>), 필운동(<i>弼雲洞</i>), 내자동(<i>內資洞</i>), 통의동(<i>通義洞</i>), 적선동(<i>積善洞</i>), 세종로(<i>世宗路</i>), 도렴동(<i>都染洞</i>), 당주동(<i>唐珠洞</i>), 내수동(<i>內需洞</i>), 신문로1.2가(<i>新門路1.2街</i>)</td>
</tr>
<tr>
<td>삼청동(<i>三淸洞</i>)</td>
<td>삼청동(<i>三淸洞</i>), 팔판동(<i>八判洞</i>), 안국동(<i>安國洞</i>), 소격동(<i>昭格洞</i>), 화동(<i>花洞</i>), 사간동(<i>司諫洞</i>), 송현동(<i>松峴洞</i>)</td>
</tr>
<tr>
<td>부암동(<i>付岩洞</i>)</td>
<td>부암동(<i>付岩洞</i>), 홍지동(<i>弘智洞</i>), 신영동(<i>新營洞</i>)</td>
</tr>
<tr>
<td>평창

In [49]:
gu = re.sub('[^가-힣]','',table_ls[0].find('caption').text)

In [55]:
tr_ls = table_ls[0].find('tbody').find_all('tr')

In [72]:
tr_ls

[<tr>
 <td>청운효자동
                                         (<i>淸雲孝子洞</i>)</td>
 <td>청운동(<i>淸雲洞</i>), 신교동(<i>新橋洞</i>), 궁정동(<i>宮井洞</i>), 세종로(<i>世宗路</i>), 효자동(<i>孝子洞</i>), 창성동(<i>昌成洞</i>), 통인동(<i>通仁洞</i>), 누상동(<i>樓上洞</i>), 누하동(<i>樓下洞</i>), 옥인동(<i>玉仁洞</i>)</td>
 </tr>,
 <tr>
 <td>사직동(<i>社稷洞</i>)</td>
 <td>사직동(<i>社稷洞</i>), 체부동(<i>體府洞</i>), 필운동(<i>弼雲洞</i>), 내자동(<i>內資洞</i>), 통의동(<i>通義洞</i>), 적선동(<i>積善洞</i>), 세종로(<i>世宗路</i>), 도렴동(<i>都染洞</i>), 당주동(<i>唐珠洞</i>), 내수동(<i>內需洞</i>), 신문로1.2가(<i>新門路1.2街</i>)</td>
 </tr>,
 <tr>
 <td>삼청동(<i>三淸洞</i>)</td>
 <td>삼청동(<i>三淸洞</i>), 팔판동(<i>八判洞</i>), 안국동(<i>安國洞</i>), 소격동(<i>昭格洞</i>), 화동(<i>花洞</i>), 사간동(<i>司諫洞</i>), 송현동(<i>松峴洞</i>)</td>
 </tr>,
 <tr>
 <td>부암동(<i>付岩洞</i>)</td>
 <td>부암동(<i>付岩洞</i>), 홍지동(<i>弘智洞</i>), 신영동(<i>新營洞</i>)</td>
 </tr>,
 <tr>
 <td>평창동(<i>平倉洞</i>)</td>
 <td>평창동(<i>平倉洞</i>), 구기동(<i>舊基洞</i>)</td>
 </tr>,
 <tr>
 <td>무악동(<i>毋岳洞</i>)</td>
 <td>무악동(<i>毋岳洞</i>)</td>
 </tr>,
 <tr>
 <td>교남동(<i>橋南洞</i>)</td>
 <td>교남동(<i>橋南洞</i>), 평동(<i>平洞</i>), 송월동(<i>

In [64]:
hang_nm = re.sub('[^가-힣]','',tr_ls[0].find_all('td')[0].text)

In [67]:
td_ls = tr_ls[0].find_all('td')[1].text.split(',')

In [94]:
temp = pd.DataFrame()
for k in table_ls:
    gu = re.sub('[^가-힣]','', k.find('caption').text.split('(')[0])
    
    
    tr_ls = k.find('tbody').find_all('tr')
    for j in tr_ls:
        hang_nm = re.sub('[^0-9가-힣]','', j.find_all('td')[0].text.split('(')[0])

        td_ls = j.find_all('td')[1].text.split(',')
        for i in td_ls:
            bub = re.sub('[^0-9가-힣]','',i.split('(')[0])

            temp0 = pd.DataFrame({'gu' : [gu]
                         ,'hang' : [hang_nm]
                         ,'bub' : [bub]})

            temp = temp.append(temp0)

In [95]:
temp

,gu,hang,bub
0,종로구,청운효자동,청운동
0,종로구,청운효자동,신교동
0,종로구,청운효자동,궁정동
0,종로구,청운효자동,세종로
0,종로구,청운효자동,효자동
...,...,...,...
0,강동구,성내제2동,성내동
0,강동구,성내제3동,성내동
0,강동구,길동,길동
0,강동구,둔촌제1동,둔촌동


In [92]:
temp[temp['gu'] == '광진구']

,gu,hang,bub
0,광진구,청운효자동,청운동
0,광진구,청운효자동,신교동
0,광진구,청운효자동,궁정동
0,광진구,청운효자동,세종로
0,광진구,청운효자동,효자동
...,...,...,...
0,광진구,창신제1동,창신동
0,광진구,창신제2동,창신동
0,광진구,창신제3동,창신동
0,광진구,숭인제1동,숭인동


In [76]:
td_ls

['청운동(淸雲洞)',
 ' 신교동(新橋洞)',
 ' 궁정동(宮井洞)',
 ' 세종로(世宗路)',
 ' 효자동(孝子洞)',
 ' 창성동(昌成洞)',
 ' 통인동(通仁洞)',
 ' 누상동(樓上洞)',
 ' 누하동(樓下洞)',
 ' 옥인동(玉仁洞)']

In [84]:
temp[temp['hang']=='이화동']

,gu,hang,bub
0,종로구,이화동,이화동
0,종로구,이화동,연건동
0,종로구,이화동,동숭동


In [71]:
bub_ls

['청운동', '신교동', '궁정동', '세종로', '효자동', '창성동', '통인동', '누상동', '누하동', '옥인동']